In [1]:
filename="data.tsv"
raw_text=open(filename,'r').read().lower()

In [2]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
from collections import defaultdict
 
    
# Find popular words mentioned in the dataset to determine topics of haiku
stop_words = set(stopwords.words('english')) 
punctuation = set(string.punctuation)

word_dict = defaultdict(int)
for w, prop in nltk.pos_tag(word_tokenize(raw_text)):
    w = ''.join([c for c in w if not c in punctuation])
    if w in stop_words or w =='br' or w == '':
        continue
    elif prop == 'NN':
        word_dict[w] += 1
            
popularWords = [(word_dict[x], x) for x in word_dict]
popularWords.sort()
popularWords.reverse()


In [6]:
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [7]:
def read_input(input_file):
    with open(input_file, 'rb') as f:
        for i, line in enumerate(f):
            yield gensim.utils.simple_preprocess(line)

In [9]:
doc = read_input('data.tsv')
# build vocabulary and train model
doc_list = [d for d in doc]
model = gensim.models.Word2Vec(
        doc_list,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model.train(doc_list, total_examples=len(doc_list), epochs=10)

(34740628, 50344940)

In [10]:
w1 = 'something'
model.wv.most_similar(positive=w1)

/Users/Herman/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('anything', 0.7117910385131836),
 ('nothing', 0.5874983072280884),
 ('whatever', 0.5577409267425537),
 ('things', 0.5481641292572021),
 ('someone', 0.5237470865249634),
 ('everything', 0.5168544054031372),
 ('it', 0.5111933946609497),
 ('what', 0.5087223052978516),
 ('stuff', 0.47133585810661316),
 ('snowflake', 0.45040780305862427)]

In [11]:
word_vectors = model.wv
del model
word_vectors['computer']

array([-1.2642795 , -0.51795095,  0.67204547, -0.38518867,  1.0278109 ,
        0.1024085 , -0.05360449,  0.31515282, -0.23021771, -0.7869888 ,
       -2.0556011 , -0.85081035,  0.7705266 ,  0.36268896, -1.2520938 ,
        2.0300279 ,  0.06054727, -1.1553769 ,  1.5840745 , -0.02483731,
        2.2633948 ,  0.5388736 ,  0.53490126,  0.9022774 , -0.6825171 ,
       -0.4778896 ,  1.7844342 , -0.7965914 , -1.1970766 ,  0.6596457 ,
        0.53584594,  0.41376653,  0.36950696, -1.5192869 , -0.7544989 ,
        0.9097853 , -1.4788796 ,  2.0131588 , -0.6739045 , -0.05928412,
       -0.45873353, -0.71868694, -1.7373323 , -0.7812943 ,  1.2414864 ,
       -0.7297306 ,  0.24165006, -0.6198493 ,  2.5534575 , -1.7465446 ,
       -0.93307346, -1.3158338 , -0.23063755,  1.1519043 ,  1.2046088 ,
       -0.16371915, -0.95074016, -1.1438292 ,  0.59983766, -1.2115061 ,
       -0.32691258, -0.356442  ,  3.9180996 ,  0.7040092 , -0.5911093 ,
       -0.64512473, -1.9769596 , -2.3769882 ,  0.898313  , -0.82

In [12]:
topics = ['time', 'game', 'life', 'money', 'person']

for t in topics:
    print(word_vectors.n_similarity(['key'], [t]))

0.13183185
0.27820852
0.045675382
0.09925018
-0.023616277


In [38]:
import requests
from bs4 import BeautifulSoup

def count_word_syllables(word):
    url = 'http://www.syllablecount.com/syllables/'+word
    body = requests.get(url).text
    soup = BeautifulSoup(body, 'lxml')
    
    res = str(soup.find_all(attrs={'class':'col1'})).split('How many syllables are in')[1].split('syllable')[0]
    return int(res.split('>')[1])

In [39]:
def count_syllables(sentence):
    words = sentence.split(' ')
    count = 0
    for w in words:
        count += count_word_syllables(w)
    return count

In [40]:
count_syllables("if you kill yourself")

5

In [31]:
def find_rhymes(word):
    url = 'http://rhymebrain.com/en/What_rhymes_with_' + word + '.html'
    body = requests.get(url).text
    soup = BeautifulSoup(body, 'lxml')
    
    res = str(soup.find_all(attrs={'class':'wordpanel'}))
    res = [x for x in res.split('>') if x.find('wordpanel') == -1]
    res = [x.split('<')[0].strip() for x in res if x != "]"]
    return res

In [37]:
res = find_rhymes('friends')

['ends', 'lends', 'tends', 'trends', 'sends', 'spends', 'bends', 'blends', 'amends', 'attends', 'ascends', 'offends', 'depends', 'extends', 'descends', 'intends', 'defends', 'pretends', 'commends', 'suspends', 'dividends', 'recommends', 'contends', 'transcends', 'apprehends', 'comprehends', 'rents', 'cents', 'tents', 'scents', 'vents', 'dents', 'events', 'temps', 'laments', 'ferments', 'resents', 'prevents', 'intents', 'descents', 'extents', 'invents', 'consents', 'represents', 'presidents', 'penitents', 'discontents', 'malcontents', 'lens', 'threads', 'reds', 'shreds', 'ens', 'sleds', 'heads', 'legs', 'beds', 'eggs', 'heirs', 'realms', 'spreads', 'tens', 'airs', 'cleanse', 'hens', 'pens', 'sheds', 'dens', 'elms', 'flares', 'gens', 'welds', 'yells', 'breads', 'dreads', 'elves', 'glens', 'treads', 'cells', 'says', 'tears', 'tells', 'bears', 'pairs', 'prayers', 'shares', 'stairs', 'bells', 'cares', 'chairs', 'shells', 'stems', 'theirs', 'wells', 'hairs', 'sells', 'selves', 'shelves', 'sp